In [2]:
from sympy import * 
from sympy.printing import print_ccode
init_printing(use_latex='mathjax')
from IPython.display import display_markdown

In [3]:
def display_vars(*args):
    if isinstance(args[0], str):
        args = (args,)
    display_markdown("$$" + ',\;'.join(name + ' = ' + latex(var) for name, var in args) + "$$", raw=True)

In [79]:
state  = Matrix(symbols('h, p, T, \dot{T}', real=True))
predict = Matrix(symbols('h_\omega, p_\omega, T_\omega, \dot{T}_\omega'))
predict_measure = Matrix(symbols('p_m, T_m'))

display_vars(("x", state),
             ("\hat{x}", predict),
             ("\hat{z}", predict_measure))

$$x = \left[\begin{matrix}h\\p\\T\\\dot{T}\end{matrix}\right],\;\hat{x} = \left[\begin{matrix}h_\omega\\p_\omega\\T_\omega\\\dot{T}_\omega\end{matrix}\right],\;\hat{z} = \left[\begin{matrix}p_{m}\\T_{m}\end{matrix}\right]$$

In [80]:
p0, g, m, r = symbols('p_0, g, m, r', real=True)

In [100]:
dt = Symbol("\Delta t", real=True)
predict_fun = Matrix([state[0], p0 * exp(-state[0]*g*m/(r*state[2])), state[2] + dt*state[3], state[3]])
A = predict_fun.jacobian(state)
display_vars(("f(x)", predict_fun),
             ("A", A))

$$f(x) = \left[\begin{matrix}h\\p_{0} e^{- \frac{g h m}{T r}}\\T + \Delta t \dot{T}\\\dot{T}\end{matrix}\right],\;A = \left[\begin{matrix}1 & 0 & 0 & 0\\- \frac{g m p_{0}}{T r} e^{- \frac{g h m}{T r}} & 0 & \frac{g h m p_{0}}{T^{2} r} e^{- \frac{g h m}{T r}} & 0\\0 & 0 & 1 & \Delta t\\0 & 0 & 0 & 1\end{matrix}\right]$$

In [101]:
alpha = symbols('alpha', real=True)
predict_measure = Matrix(symbols('\hat{p}_m, \hat{t}_m'))
measure_fun = Matrix([predict[1] + alpha*predict[2],
                      predict[2]])

H = measure_fun.jacobian([predict])
display_vars("H", H)

$$H = \left[\begin{matrix}0 & 1 & \alpha & 0\\0 & 0 & 1 & 0\end{matrix}\right]$$

In [102]:
print(ccode(A, MatrixSymbol('out', 4, 4)).replace('\\Delta t', 'dt').replace('p_0', 'p0').replace('T', 't'))

out[0] = 1;
out[1] = 0;
out[2] = 0;
out[3] = 0;
out[4] = -g*m*p0*exp(-g*h*m/(t*r))/(t*r);
out[5] = 0;
out[6] = g*h*m*p0*exp(-g*h*m/(t*r))/(pow(t, 2)*r);
out[7] = 0;
out[8] = 0;
out[9] = 0;
out[10] = 1;
out[11] = dt;
out[12] = 0;
out[13] = 0;
out[14] = 0;
out[15] = 1;
